# Evolver Loop 11 - LB Feedback Analysis

## Submission Result
- **CV Score**: 70.6305
- **LB Score**: 70.6305 (PERFECT CALIBRATION!)
- **Target**: 68.919154
- **Gap**: 1.711 points (2.42%)

## Key Insight from Web Research
Top teams achieving sub-69 scores use THREE different packers:
1. **Chebyshev-distance square-packing** - regular grid
2. **Smart Scanline linear packer** - sweep line, insert at first feasible position
3. **2-tree alternating lattice** - pairs of trees alternating up/down

They run ALL THREE for each N=1-200 and pick the smallest sn.

In [1]:
import pandas as pd
import numpy as np
import json

# Load current best solution
current_best = pd.read_csv('/home/code/exploration/datasets/saspav_best.csv')
print(f"Current best solution: {len(current_best)} rows")
print(current_best.head())

Current best solution: 20100 rows
      id                       x                      y  \
0  001_0  s-48.19608619421424578  s58.77098461521422479   
1  002_0    s0.15409706962136430  s-0.03854074269478543   
2  002_1   s-0.15409706962136430  s-0.56145925730521462   
3  003_0    s1.12365581614030097   s0.78110181599256301   
4  003_1    s1.23405569584216002   s1.27599950066375900   

                      deg  
0   s45.00000000000000000  
1  s203.62937773065684155  
2   s23.62937773065679181  
3  s111.12513229289299943  
4   s66.37062226934300213  


In [2]:
# Analyze per-N scores to find where we have the most room for improvement
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def load_trees(n, df):
    group_data = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row['x']).lstrip('sx')
        y = str(row['y']).lstrip('sy')
        deg = str(row['deg']).lstrip('sd')
        trees.append(ChristmasTree(x, y, deg))
    return trees

def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

print("Functions defined")

Functions defined


In [3]:
# Calculate per-N scores
per_n_scores = {}
for n in range(1, 201):
    trees = load_trees(n, current_best)
    per_n_scores[n] = calculate_score(trees)

# Find N values with highest scores (most room for improvement)
sorted_scores = sorted(per_n_scores.items(), key=lambda x: x[1], reverse=True)

print("Top 20 N values with highest per-N scores (most room for improvement):")
print("="*60)
for n, score in sorted_scores[:20]:
    print(f"N={n:3d}: score = {score:.6f}")

print("\nTotal score:", sum(per_n_scores.values()))

Top 20 N values with highest per-N scores (most room for improvement):
N=  1: score = 0.661250
N=  2: score = 0.450779
N=  3: score = 0.434745
N=  5: score = 0.416850
N=  4: score = 0.416545
N=  7: score = 0.399897
N=  6: score = 0.399610
N=  9: score = 0.387415
N=  8: score = 0.385407
N= 15: score = 0.376978
N= 10: score = 0.376630
N= 21: score = 0.376451
N= 20: score = 0.376057
N= 22: score = 0.375258
N= 11: score = 0.374924
N= 16: score = 0.374128
N= 26: score = 0.373997
N= 12: score = 0.372724
N= 13: score = 0.372294
N= 25: score = 0.372144

Total score: 70.6304783244917


In [4]:
# Analyze score distribution by N ranges
ranges = [
    (1, 10, "Small (1-10)"),
    (11, 50, "Medium (11-50)"),
    (51, 100, "Large (51-100)"),
    (101, 150, "Very Large (101-150)"),
    (151, 200, "Huge (151-200)")
]

print("Score contribution by N range:")
print("="*60)
for start, end, name in ranges:
    range_score = sum(per_n_scores[n] for n in range(start, end+1))
    range_pct = range_score / sum(per_n_scores.values()) * 100
    print(f"{name:20s}: {range_score:.4f} ({range_pct:.1f}%)")

Score contribution by N range:
Small (1-10)        : 4.3291 (6.1%)
Medium (11-50)      : 14.7050 (20.8%)
Large (51-100)      : 17.6170 (24.9%)
Very Large (101-150): 17.1364 (24.3%)
Huge (151-200)      : 16.8430 (23.8%)


In [5]:
# Calculate theoretical minimum scores
# Tree bounding box at angle=45: ~0.813 x 0.813 (roughly square)
# Tree polygon area: ~0.3 (approximate)

tree = ChristmasTree('0', '0', '45')
tree_area = tree.polygon.area / (1e15 ** 2)
print(f"Tree polygon area at angle=45: {tree_area:.6f}")

# For N trees packed perfectly, minimum side = sqrt(N * tree_area / packing_efficiency)
# Typical packing efficiency for irregular polygons: 60-70%

print("\nTheoretical analysis:")
print("="*60)
for n in [1, 10, 50, 100, 150, 200]:
    current = per_n_scores[n]
    current_side = np.sqrt(current * n)
    
    # Theoretical minimum with 70% packing efficiency
    theoretical_side = np.sqrt(n * tree_area / 0.70)
    theoretical_score = theoretical_side ** 2 / n
    
    gap = current - theoretical_score
    gap_pct = gap / current * 100
    
    print(f"N={n:3d}: current={current:.4f}, theoretical={theoretical_score:.4f}, gap={gap:.4f} ({gap_pct:.1f}%)")

Tree polygon area at angle=45: 0.245625

Theoretical analysis:
N=  1: current=0.6612, theoretical=0.3509, gap=0.3104 (46.9%)
N= 10: current=0.3766, theoretical=0.3509, gap=0.0257 (6.8%)
N= 50: current=0.3608, theoretical=0.3509, gap=0.0099 (2.7%)
N=100: current=0.3434, theoretical=0.3509, gap=-0.0075 (-2.2%)
N=150: current=0.3371, theoretical=0.3509, gap=-0.0138 (-4.1%)
N=200: current=0.3375, theoretical=0.3509, gap=-0.0133 (-4.0%)


In [6]:
# Key insight: The gap to target (1.711 points) is 2.42% of current score
# This is a SIGNIFICANT gap that cannot be closed by micro-optimization

# What we've tried that DIDN'T work:
# 1. Random restart SA - NO improvements
# 2. Exhaustive search for N=1,2 - baseline already optimal
# 3. Genetic algorithm - NO improvements
# 4. Tessellation SA - WORSE than baseline
# 5. Deletion cascade - NO improvements
# 6. Ensemble from 25 sources - hit ceiling at 70.630478

# What we HAVEN'T tried:
# 1. Chebyshev-distance square-packing
# 2. Smart Scanline linear packer
# 3. 2-tree alternating lattice
# 4. Running C++ optimizer with PER-N parameters (not total iterations)

print("Summary of approaches:")
print("="*60)
print("TRIED (no improvement):")
print("  - Random restart SA")
print("  - Exhaustive search N=1,2")
print("  - Genetic algorithm")
print("  - Tessellation SA")
print("  - Deletion cascade")
print("  - Ensemble from 25 sources (hit ceiling)")
print()
print("NOT TRIED:")
print("  - Chebyshev-distance square-packing")
print("  - Smart Scanline linear packer")
print("  - 2-tree alternating lattice")
print("  - Per-N optimization with C++ optimizer")

Summary of approaches:
TRIED (no improvement):
  - Random restart SA
  - Exhaustive search N=1,2
  - Genetic algorithm
  - Tessellation SA
  - Deletion cascade
  - Ensemble from 25 sources (hit ceiling)

NOT TRIED:
  - Chebyshev-distance square-packing
  - Smart Scanline linear packer
  - 2-tree alternating lattice
  - Per-N optimization with C++ optimizer


In [7]:
# The key insight from web research:
# Top teams run THREE different packers for each N and pick the best
# This is fundamentally different from what we've been doing

# Our current approach: Ensemble from public sources
# Top team approach: Generate NEW solutions with multiple algorithms

# The gap (1.711 points) requires:
# - Average improvement of 0.00856 per N value
# - Or significant improvements on a few high-impact N values

print("Gap analysis:")
print("="*60)
print(f"Current total: 70.630478")
print(f"Target: 68.919154")
print(f"Gap: 1.711324")
print(f"Average improvement needed per N: {1.711324/200:.6f}")
print()
print("If we improve only the top 20 N values:")
print(f"  Average improvement needed: {1.711324/20:.6f} per N")
print()
print("If we improve only the top 50 N values:")
print(f"  Average improvement needed: {1.711324/50:.6f} per N")

Gap analysis:
Current total: 70.630478
Target: 68.919154
Gap: 1.711324
Average improvement needed per N: 0.008557

If we improve only the top 20 N values:
  Average improvement needed: 0.085566 per N

If we improve only the top 50 N values:
  Average improvement needed: 0.034226 per N
